# Imports

In [245]:
from pprint import pprint
import regex as re
import html
from bs4 import BeautifulSoup
import requests
from requests.exceptions import Timeout
import pandas as pd
from helium import *
from selenium.webdriver import DesiredCapabilities

import download_imgs # used in getAsyncImgFunctions()

import multiprocessing
from multiprocessing import Process
import pickle
import os
import json
import time
from datetime import datetime

# Global Variables & Functions

In [202]:
dataDir = 'dataset/'
imgTypes = list(filter(lambda v: re.match('\d{1,3}\. ', v), os.listdir(dataDir)))
imgTypes

['0. normal',
 '1. fmemes',
 '2. ememes',
 '3. eSocialMedia',
 '4. fFbPosts',
 '5. fTwtrPosts',
 '6. fTxtMssgs',
 '7. eGreetingAndMisc']

In [203]:
def pklSave(contentToBeSaved, fullPath):
    with open(fullPath, 'wb') as f:
        pickle.dump(contentToBeSaved, f)

def pklLoad(fullPath):
    with open(fullPath, 'rb') as f:
        content = pickle.load(f)
    return content

def forceLoadPickle(path, dtype = 'dict'):
    try:
        content = pklLoad(path)
        return content
    except Exception as e:
        if dtype == 'list':
            pklSave([], path)
            return []
        else:
            pklSave({}, path)
            return {}

## Setting up images and Labels' dictionaries

In [204]:
imgTypeToNum = {}
for  i, type in enumerate(imgTypes):
    imgTypeToNum[type] = i
imgTypeToNum

{'0. normal': 0,
 '1. fmemes': 1,
 '2. ememes': 2,
 '3. eSocialMedia': 3,
 '4. fFbPosts': 4,
 '5. fTwtrPosts': 5,
 '6. fTxtMssgs': 6,
 '7. eGreetingAndMisc': 7}

In [205]:
pklSave(imgTypeToNum, dataDir+"imgTypeToNum.pickle") # to be later used in another .ipynb file

# Scraping Foreign Memes

## From [Kaggle's Meme Generator Dataset](https://www.kaggle.com/datasets/electron0zero/memegenerator-dataset)

### Getting memes' URLs

In [206]:
fmemesCsvLinks = pd.read_csv("dataset/memegenerator.csv", usecols=["Meme ID", "Base Meme Name"]).squeeze() # convert to Series
fmemesCsvLinks.head()

,Meme ID,Base Meme Name
0,10509464,Spiderman Approves
1,12285257,Alright Then Business Kid
2,20612245,Archer
3,20614628,Futurama Fry
4,24194267,One Does Not Simply


Upon inspection, we want to get the original template of memes, not its varientswe'll find that to fetch a page of only the meme image, the URL has to be in this format <br>
`https://memegenerator.net/img/instances/XX.jpg` <br>
Where `XX` is the unique id of the image on [Meme Generator](https://memegenerator.net/) <br>
Thus, we will use the `Meme ID` column

In [207]:
fmemesIds = fmemesCsvLinks['Meme ID']
fmemesIds.head()

0    10509464
1    12285257
2    20612245
3    20614628
4    24194267
Name: Meme ID, dtype: object

In [208]:
pd.set_option('max_colwidth', 70)
fmemesLinks = 'https://memegenerator.net/img/instances/' + fmemesIds + '.jpg'
fmemesLinks.head()

0    https://memegenerator.net/img/instances/10509464.jpg
1    https://memegenerator.net/img/instances/12285257.jpg
2    https://memegenerator.net/img/instances/20612245.jpg
3    https://memegenerator.net/img/instances/20614628.jpg
4    https://memegenerator.net/img/instances/24194267.jpg
Name: Meme ID, dtype: object

In [209]:
pd.set_option('max_colwidth', 40)

### Downloading the foreign images

In [210]:

def getAsyncImgFunctions(imgsLinks, imgType, interval, offset=0):
    fns = []
    i = 0
    check = True
    while i < len(imgsLinks):
        fns.append((download_imgs.downloadImgs, (imgType, imgsLinks, i, i+interval, offset)))
        i += interval
    return fns

In [211]:
fns = getAsyncImgFunctions(fmemesLinks, imgTypes[1], 2000)

In [212]:
# Using runInParallel() below, we will run xx functions parallely to fetch data
print(len(fns))

29


In [213]:
def runInParallel(fns):
  proc = []
  for fn in fns:
    p = Process(target=fn[0], args=(fn[1]))
    p.start()
    proc.append(p)
  for p in proc:
    p.join()

In [214]:
#if __name__ == '__main__':
    #runInParallel(fns)

# Scraping Egyptian Memes

## From Imgur using this [Facebook post]()

In [215]:
fbPostContent = "Templates HQ - المنتدى ( ألبومات تمبلتس _ مستلزمات الكوميك _ شروحات _ افضل المواقع _ مستلزمات التصميم _ ايموشنز _ وشوش ) : plus.google.com/u/0/communities/108675057016512986137 ____ ألبومات تمبلتس :(تمبلتس | مقصوصه ، افلام ، مسلسلات ، مشاهد ، برامج ) ___________ .......... محمد هنيدى .......... رمضان مبروك ابو العالمين حمودة | imgur.com/a/HxfPF اسماعيلية رايح جى | imgur.com/a/i5Msi فول الصين العظيم | imgur.com/a/C8OLI جائنا البيان التالى | imgur.com/a/r4X66 صاحب صاحبه | imgur.com/a/ilzyz يا انا يا خالتى | imgur.com/a/sTx1q امير البحار | imgur.com/a/mRYs0 وش اجرام | imgur.com/a/lhDIg .......... محمد سعد .......... اللي بالي بالك | imgur.com/a/mMJkA اللمبى‏ | imgur.com/a/kNwD8 بوحه | imgur.com/a/nqdmP .......... عادل امام .......... عريس من جهة امنيه | imgur.com/a/h4pA7 السفارة فى العمارة | imgur.com/a/Y519O سلام يا صاحبى | imgur.com/a/XsucN زهايمر | imgur.com/a/iLUE0 .......... احمد مكى .......... لا تراجع ولا استسلام | imgur.com/a/Ri6QP الكبير اوى | imgur.com/a/fUQNf طير انت | imgur.com/a/NPUrK .......... شيكو و هشام ماجد و أحمد فهمي .......... سمير وشهير وبهير | imgur.com/a/UeRnH حملة فريزر | imgur.com/a/bKB5D بنات العم | imgur.com/a/wMQTL .......... كريم عبد العزيز .......... حرامية في تايلاند‏ | imgur.com/a/MxxS6 فى محطة مصر | imgur.com/a/xxLOL الباشا تلميذ | imgur.com/a/VS6Ui ابو على | imgur.com/a/GdBOz .......... احمد رزق و احمد عيد .......... اوعى وشك | imgur.com/a/tw0Zk فيلم ثقافى‏ | imgur.com/a/AIgjT .......... احمد ادم.......... الرجل الابيض المتوسط | imgur.com/a/OKOME معلش إحنا بنتبهدل | imgur.com/a/VFUyL .......... احمد حلمى .......... عسل اسود | imgur.com/a/mXJkz على جثتى | imgur.com/a/1xzZT كده رضا | imgur.com/a/dxfdi .......... حماده هلال .......... الهرم الرابع | imgur.com/a/wAhJp عيال حبيبة |imgur.com/a/QpP7R غبي منه فيه | imgur.com/a/hYvug مقلب حرامية | imgur.com/a/bbTvu هى فوضى | imgur.com/a/PMlpR تيمور وشفيقه | imgur.com/a/qjcBf طباخ الريس | imgur.com/a/vIzBa كابتن هيما | imgur.com/a/X3abp حلم العمر | imgur.com/a/h1w4F الناظر | imgur.com/a/UQWH7 ايظن | imgur.com/a/G4q1vA2 Euc | imgur.com/a/wM1rrTE ..... تمبلتس باسم يوسف | imgur.com/a/QoM3E تمبلتس افلام اجنبى | imgur.com/a/m2anE ـــــــــــــــــــــــــــــــــــــــ البوم الكيف (مشهد العزاء) | imgur.com/a/ysymZ البوم عايز حقى (مشهد هواء هواء) | imgur.com/a/rBvFC البوم مرتضى (انا معايا دورى وصوير و 4 كاس) | imgur.com/a/Mf9OK البوم لخمة راس (هو انت ياد مش بتتكيف غير لم تضرب)| imgur.com/a/5v0GK البوم ابو العربى ( مشهد الكباريه ) | imgur.com/a/Ga7kGAY البوم الناظر (اوعى يغرك جسمك) | imgur.com/a/hwjFu ألبوم لن اعيش فى جلباب ابى | imgur.com/a/K2bosbc ـــــــــــــــــــــــــــــــــــــــ تمبلتس اعلانات ........ اتصالات محمد رمضان 2017 | imgur.com/a/Pxuv9 فودافون 2017 | imgur.com/a/BaCF8 حديد الجارحى | imgur.com/a/4uLwR كارير | imgur.com/a/pxapm بوادى | imgur.com/a/QdWKQ ـــــــــــــــــــــــــــــــــــــــ تمبلتس مقصوصة | imgur.com/a/y1bDn ــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ ⛔ كل الألبومات بنعدلها كل يوم وبنضيف عليها كل التمبلتس الجديده | Facebook"
ememesHashes = re.findall("imgur.com/a/([A-Za-z]{5})", fbPostContent)
print(ememesHashes)

['HxfPF', 'ilzyz', 'lhDIg', 'mMJkA', 'nqdmP', 'XsucN', 'fUQNf', 'NPUrK', 'UeRnH', 'wMQTL', 'xxLOL', 'GdBOz', 'AIgjT', 'OKOME', 'VFUyL', 'mXJkz', 'dxfdi', 'wAhJp', 'hYvug', 'bbTvu', 'PMlpR', 'qjcBf', 'vIzBa', 'ysymZ', 'rBvFC', 'hwjFu', 'pxapm', 'QdWKQ']


### Using [Imgur's API](https://api.imgur.com/)

In [216]:
client_id = 'your-data-here'
client_secret = 'your-data-here'

In [217]:
headers = {
    "Content-Type": "text",
    'Authorization' : "Client-ID " + client_id
}

In [218]:
jsn = json.loads(requests.get('https://api.imgur.com/3/album/i5Msi/images', headers=headers).text)
pprint(jsn)


{'data': {'error': 'Invalid client_id',
          'method': 'GET',
          'request': '/3/album/i5Msi/images'},
 'status': 403,
 'success': False}


Note that initially (before I removed cliend_id string) the JSON response was like this: <br><br>
<img src='project_media/0. imgur_api_response.png' width=350>

In [219]:
ememesLinks = []
def getImgurLinks():
    for hash in ememesHashes:
        imgsDetailsJson = json.loads(requests.get(f'https://api.imgur.com/3/album/{hash}/images', headers=headers).text)
        for img in imgsDetailsJson['data']:
            ememesLinks.append(img['link'])

In [220]:
#getImgurLinks()

In [221]:
#pklSave(ememesLinks, dataDir+"ememesImgurLinks.pickle") # store the links, in order not to keep accessing imgur's API each time we run the cells

In [222]:
#ememesLinks = pklLoad(dataDir+"ememesImgurLinks.pickle")
#fns = getAsyncImgFunctions(ememesLinks, imgTypes[2], len(ememesLinks)//50) #last argument means this: if #links to download are 10,000 then there will be 50 functions, each will take from link i to link i + 10,000/50 (which is 200)  
#if __name__ == '__main__':
    #runInParallel(fns)

## From Facebook's public meme pages

In [223]:
fbLinks = [ # These are just some Egyptian pages that were gathered
    "https://www.facebook.com/%D9%85%D9%8A%D9%85%D8%B2-%D9%84%D8%A7-%D9%81%D8%A7%D8%A6%D8%AF%D8%A9-%D9%85%D9%86%D9%87%D8%A7-%D9%85%D8%AB%D9%84-%D8%AD%D9%8A%D8%A7%D8%AA%D9%83-290489111460684/photos",
    "https://www.facebook.com/MemesYard/photos/?ref=page_internal",
    "https://www.facebook.com/memes.Stolen.1.0/photos/?ref=page_internal",

    "https://www.facebook.com/%D8%A8%D9%86%D8%B3%D8%B1%D9%82-%D9%85%D9%8A%D9%85%D8%B2-%D9%88%D9%83%D9%88%D9%85%D9%8A%D9%83-%D8%B9%D8%B4%D8%A7%D9%86-%D9%85%D8%B4-%D8%A8%D9%86%D8%B9%D8%B1%D9%81-%D9%86%D8%B9%D9%85%D9%84-1735677470065180/photos/?ref=page_internal",
    "https://www.facebook.com/profile.php?id=100064850389099&sk=photos",
    "https://www.facebook.com/%D9%85%D9%8A%D9%85%D8%B2-%D9%84%D9%88%D8%B1%D8%AF-%D9%82%D9%85%D8%AF-100182408618014/photos/?ref=page_internal",
    
    "https://www.facebook.com/%D9%85%D9%8A%D9%85%D8%B2-%D9%85%D8%B5%D8%B1%D9%8A%D9%87-101966151287716/photos/?ref=page_internal",
    "https://www.facebook.com/%D9%85%D9%8A%D9%85%D8%B2-%D9%85%D8%B4-%D9%87%D9%8A%D9%81%D9%87%D9%85%D9%87%D8%A7-%D8%A7%D9%84%D9%86%D9%88%D8%B1%D9%85%D9%8A%D8%B2-%D8%B9%D8%B4%D8%A7%D9%86-%D9%86%D9%88%D8%B1%D9%85%D9%8A%D8%B2-833501407023909/photos/?ref=page_internal",
    "https://www.facebook.com/arabicclassicalartmemes/photos/?ref=page_internal",
    
    "https://web.facebook.com/True.Memes.Comics/photos/?ref=page_internal",
    "https://web.facebook.com/societyforsarcasm/photos/?ref=page_internal",
    "https://web.facebook.com/memes.officil/photos",
]

In [224]:
def scrollTillEnd(browser):
    """ 
    This function uses Helium (or Selenium) browser to scroll 
    Through a page that dynamically loads content in order to 
    fully load the html in browser.page_source, example:
    https://www.facebook.com/progressofyear/photos
    """
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    lenOfPage = browser.execute_script("var lenOfPage=document.body.scrollHeight; return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight); window.scrollBy(0,-200);") # "-200" scrolls up a little, so that FB realizes that the user is approaching the end, thus load more content
        lenOfPage = browser.execute_script("var lenOfPage=document.body.scrollHeight; return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

In [225]:
def fbPageName(htmlText):
    """
    Fetches name of a FB page, which upon inspection is usally in 
    the first <h1>, <h2>, <span> of <h1> or <span> of <h2> 
    """
    headerOfPageTitle = re.search('<h[12] .+?(?=/h[12]>)', htmlText)
    if (headerOfPageTitle is None):
        return ""
    headerOfPageTitle = headerOfPageTitle.group() #group() to get the entired matched string out of the regex "match" object
    spanText = re.search('<span>(.*)</span>', headerOfPageTitle)
    if (spanText is not None):
        pageName = spanText.group(1).strip() # group(1) to return what is between parentheses
    else:
        pageName = re.search('>(.*)<', headerOfPageTitle).group(1).strip()
    return pageName

In [226]:
def fbExtractImgsLinks(htmlText):
    """
    Extracts links to images in the "photos" section of a FB page,
    which usually starts from the label "Photos" or "All photos"
    """
    try:
        htmlStartingFromAllPhotos = re.findall(r'>(?:All )?[pP]hotos.*', htmlText)[0] # Regex to get pages starting from "All photos" or "photos", but it has to be preceded by ">", in order not to match "photos" in random URLs
    except Exception as e:
        print(e)
        return []
    imgLinks = re.findall(r'https://scontent[^"]+', htmlStartingFromAllPhotos)
    imgLinks = [html.unescape(link) for link in imgLinks][1:] #avoiding first image, as it might be the page's logo
    return imgLinks

def fbExtractImgsLinksOld(heliumBrowser): # didn't use this, as regex is faster than selenium methods 
    imgTags = heliumBrowser.find_elements_by_tag_name('img')
    imgLinks = []
    for link in imgTags:
        if link.get_attribute('src') is not None:
            imgLinks.append(link.get_attribute('src'))
    imgLinks = [html.unescape(link) for link in imgLinks][1:] #avoiding first image, as it might be the page's logo
    return imgLinks

In [227]:
def fbPageImgScraper(pagesLinks, heliumBrowser, singlePagePickleName, allPagesPickleName, pagesNamesPickle, prevObtainedPages={}):
    """
    Uses fbPageName() to get page name from each FB page in the "pagesLinks" list,
    and  Uses fbExtractImgsLinks() to get all images' links from these pages.
    Note that "prevObtainedPages" is a dictionary of the already fetched pages, so that 
    if this function is executed again, it won't scrape previously scraped pages.
    At the end, it saves all the links and page names in pickle files.
    """
    allLinks = []
    prevObtainedPages = {}
    for i in range(len(pagesLinks)):
        heliumBrowser.get(pagesLinks[i])
        pageName = fbPageName(heliumBrowser.page_source)
        if pageName == "":
            print("page not found...")
            continue
        if (pageName in prevObtainedPages):
            print("This page has already been scraped")
            continue
        time.sleep(1) # code from here till the try/except is to close the popup that sometimes appear when scrolling down a fb page (while not signed in)
        press(PAGE_DOWN) 
        press(PAGE_DOWN)
        time.sleep(1)
        try:
            click('Close')
        except Exception as e:
            print("pop-up didn't  show, moving on...")
        print("Scrolling page:", pageName)
        scrollTillEnd(heliumBrowser)
        imgLinks = fbExtractImgsLinks(heliumBrowser.page_source)
        allLinks.extend(imgLinks) #extend() expands the elements of an iterable
        pklSave(imgLinks, f"{dataDir}{singlePagePickleName}{i}.pickle")
        prevObtainedPages[pageName] = (i, pagesLinks[i])
        if i == len(pagesLinks)-1:
            print("Finished scraping pages!")
        else:
            print("Finished, scraping next link...", end='\n\n')
    pklSave(allLinks, f"{dataDir}{allPagesPickleName}.pickle")
    pklSave(prevObtainedPages, f"{dataDir}{pagesNamesPickle}.pickle")

In [228]:
# Side note: as you see, the function will sequentially check the fb pages, that's because my attempt of checking the pages asynchronously in "fb_page_img_scraper.py" have failed
#browser = start_chrome(headless=False)
#fbPageImgScraper(fbLinks, browser, "ememesLinksfbPage", "allFbPagesEmemes", "fbPagesUsed")
#browser.quit()


Note that when `fbPageImgScraper()` was initially called, it saved the links in .pickle files <br>and printed the name of the pages, such as these last 4: <br><br>
<img src='project_media/1. fbEMemes.png' width=350>

### Downloading the memes into "`2. ememes`" directory

In [229]:
# Note 1: I've previously saved 1247 memes (from imgur) in that folder
# I want to save files starting from the index after the last saved photo "ememes0001246.jpg"
# So it will start from "ememes0001247.jpg" 
# Note 2: The current output of this cell is obviously not 1247, because by now I've already added the rest of the images
prevFiles = os.listdir(f"{dataDir}{imgTypes[2]}")
if len(prevFiles) > 0:
    lastFileName = prevFiles[-1]
    dotLoc = lastFileName.find('.')
    offset = int(lastFileName[dotLoc-7:dotLoc])+1
    print(offset)

16996


In [230]:
allEmemesLinks = pklLoad(f"{dataDir}allFbPagesEmemes.pickle")
prevObtainedPages = pklLoad(f"{dataDir}fbPagesUsed.pickle")
print("Total Egyptian memes scraped from FB:", len(allEmemesLinks))
print("Total FB pages scraped:", len(prevObtainedPages))
# print("Saving images to '2. ememes'")
# fns = getAsyncImgFunctions(allEmemesLinks, imgTypes[2], len(allEmemesLinks)//50, offset)
# if __name__ == '__main__':
#     runInParallel(fns)

Total Egyptian memes scraped from FB: 15749
Total FB pages scraped: 12


# Scraping Egyptian Social Media Images
For example, screenshots from FB's posts, twitter's tweets, whatsapp's/messenger's chats <br>
Examples of each of these: <br><br>
<img src='project_media/2. eSocialMedia.png' width=600>

In [231]:
fbLinks = [ # These are just some Egyptian pages that were gathered
    'https://www.facebook.com/inbox.ertu/photos/?ref=page_internal',
    'https://www.facebook.com/Inbox.Status/photos',
]

In [232]:
#browser = start_chrome(headless=False)
#fbPageImgScraper(fbLinks, browser, "eSocialMediaLinksfbPage", "allFbPagesESocialMedia", "fbESocialMediaPagesUsed")
#browser.quit()
# to-do: run the above lines ^^

In [233]:
#allESocialMediaLinks = pklLoad(f"{dataDir}allFbPagesESocialMedia.pickle")
#prevObtainedPages = pklLoad(f"{dataDir}fbESocialMediaPagesUsed.pickle")
#fns = getAsyncImgFunctions(allEmemesLinks, imgTypes[3], len(allESocialMediaLinks)//50)
#if __name__ == '__main__':
#    runInParallel(fns)

# Scraping Foreign Social Media Images
For example, screenshots from FB's posts, twitter's tweets, whatsapp's/messenger's chats <br>
Examples: <br><br>
<img src='project_media/3. fFb.png' width=300>
<img src='project_media/4. fTwtr.png' width=300>
<img src='project_media/5. fWhats.png' width=300>

Side note: since all of the cells below are commented <br> 
(because they've already saved the links to pickle files, so no need to re-run them) <br>
here is the output of the `getSubredditImgsLinks()` function used on `bestoftwitter` subreddit: <br><br>
<img src='project_media/6. getSubredditImgsLinks() on twtr.png' width=400>

In [234]:
redditLinks = [ # some links that were gathered
    'https://www.reddit.com/r/FacebookCringe/',
    'https://www.reddit.com/r/bestoftwitter/',
    'https://www.reddit.com/r/texts/',
    'https://www.reddit.com/r/GoodFakeTexts/',
    'https://www.reddit.com/r/Badfaketexts/'
]

In [246]:
def getPushshiftData(after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?size=1000&is_video=false&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    try:
        r = requests.get(url, timeout=(10, 20))
    except Timeout as t:
        print('no internet connection, or api.pushift.io is down')
        print(t)
    data = json.loads(r.text)
    return data['data']

In [254]:
def getSubredditCreationDate(browser):
    dateStr = re.search('Created .+?>([^<]*)', browser.page_source)
    while dateStr is None:
        time.sleep(2)
        dateStr = re.search('Created .+?>([^<]*)', browser.page_source)
    try:
        return dateStr.group(1)
    except Exception as e:
        return "Jan 1, 2000" # a random old date

In [248]:
def getUnixCode(date):
    return int(time.mktime(date.timetuple()))
def getDate(dateStr):
    return datetime.strptime(dateStr, '%b %d, %Y').date()
def imgLinkParser(link):
    if re.match('.*(\.jpg|\.png|\.jpeg)', link):
        return link
    else:
        return ""

In [249]:
def getSubredditImgsLinks(subLink, browser, pickleName):
    redditImgLinks = []
    visitedSubreddits = {}
    browser.get(subLink)
    subName = re.search('r/([^/]*)', subLink).group(1)
    try:
        visitedSubreddits = pklLoad(f'{dataDir}visitedSubreddits.pickle')
        if subName in visitedSubreddits:
            print('This subreddit has already been scraped before!')
            return
    except:
        pass
    startDate = getUnixCode(getDate(getSubredditCreationDate(browser)))
    endDate = getUnixCode(datetime.now())
    data = getPushshiftData(
        after = startDate, 
        before = endDate, 
        sub = subName
        )
    # Will run until all posts have been gathered 
    # from the 'after' date up until before date
    while len(data) > 0:
        for submission in data:
            link = imgLinkParser(submission['url'])
            if (len(link) > 0):
                redditImgLinks.append(link)
        print("Current #links:", len(redditImgLinks))
        print('Current last img link:', redditImgLinks[-1])
        startDate = data[-1]['created_utc']
        print('It\'s creation date:')
        print(datetime.utcfromtimestamp(int(startDate)).strftime('%Y-%m-%d %H:%M:%S')) # convert from unix timestamp to readable date format
        data = getPushshiftData(startDate, endDate, subName) # Calls getPushshiftData() with the created date of the last submission
    pklSave(redditImgLinks, f'{dataDir}{pickleName}.pickle')
    visitedSubreddits[subName] = True
    pklSave(visitedSubreddits, f'{dataDir}visitedSubreddits.pickle')
    return redditImgLinks

## Scraping screenshots of FB posts

In [240]:
# capabilities = DesiredCapabilities.CHROME
# capabilities["pageLoadStrategy"] = "none"
# browser = start_chrome(headless=True, capabilities=capabilities)
# redditImgLinks = getSubredditImgsLinks(redditLinks[0], browser, "fFbPostsFromReddit")
# browser.quit()

Current #links: 107
Current last img link: https://i.redd.it/ld3id265w68z.jpg
It's creation date:
2017-07-07 19:47:11
Current #links: 221
Current last img link: https://i.redd.it/lozx4okgz2lz.jpg
It's creation date:
2017-09-10 16:18:49
Current #links: 413
Current last img link: https://i.redd.it/vf09ai7dobf01.jpg
It's creation date:
2018-02-10 05:36:05
Current #links: 627
Current last img link: https://i.redd.it/w2n69pi07ub11.jpg
It's creation date:
2018-07-24 06:05:03


KeyboardInterrupt: 

In [241]:
redditImgLinks = pklLoad(f'{dataDir}fFbPostsFromReddit.pickle')
print(len(redditImgLinks))

3128


In [ ]:
# code to save redditImgLinks into 4. fFbPosts

# comment this at the end

## Scraping screenshots of Twitter Tweets

In [242]:
# capabilities = DesiredCapabilities.CHROME
# capabilities["pageLoadStrategy"] = "none"
# browser = start_chrome(headless=True, capabilities=capabilities)
# redditImgLinks = getSubredditImgsLinks(redditLinks[1], browser, "fTwtrTweetsFromReddit")
# browser.quit()

Current #links: 173
Current last img link: https://i.redd.it/hqz9id8u3v901.jpg
It's creation date:
2018-01-13 16:31:52
Current #links: 372
Current last img link: https://i.redd.it/6y440u35xyu11.png
It's creation date:
2018-10-28 18:17:09
Current #links: 584
Current last img link: https://i.redd.it/k4mrtk9ije741.png
It's creation date:
2019-12-28 16:36:31
Current #links: 806
Current last img link: https://i.redd.it/aarszeoo9hh51.jpg
It's creation date:
2020-08-17 02:56:54
Current #links: 1022
Current last img link: https://i.redd.it/9ubs9h6ud2x61.jpg
It's creation date:
2021-05-04 08:28:30
Current #links: 1232
Current last img link: https://i.redd.it/2xoxu2ozxwi81.jpg
It's creation date:
2022-02-23 17:03:26
Current #links: 1338
Current last img link: https://i.redd.it/n5vkpyjusin91.jpg
It's creation date:
2022-09-18 07:58:16


In [255]:
redditImgLinks = pklLoad(f'{dataDir}fTwtrTweetsFromReddit.pickle')
print(len(redditImgLinks))

1338


In [ ]:
# code to save redditImgLinks into 5. fTwtrPosts

# comment this at the end

## Scraping screenshots of Chat Messages

In [250]:
# capabilities = DesiredCapabilities.CHROME
# capabilities["pageLoadStrategy"] = "none"
# browser = start_chrome(headless=True, capabilities=capabilities)
# redditImgLinks = getSubredditImgsLinks(redditLinks[2], browser, "fChats0FromReddit")
# browser.quit()

Current #links: 177
Current last img link: https://i.redd.it/wssj5qyzxf6z.jpg
It's creation date:
2017-06-28 19:51:12
Current #links: 354
Current last img link: https://i.redd.it/63mzo6c7u7kz.png
It's creation date:
2017-09-06 18:43:51
Current #links: 554
Current last img link: https://i.redd.it/859tytmyij001.jpg
It's creation date:
2017-11-27 15:58:06
Current #links: 747
Current last img link: https://i.imgur.com/5Gg5FID.jpg
It's creation date:
2018-02-04 20:41:13
Current #links: 963
Current last img link: https://i.redd.it/adpxot4wezp01.jpg
It's creation date:
2018-04-05 00:15:39
Current #links: 1160
Current last img link: https://i.redd.it/9l505ux1iay01.jpg
It's creation date:
2018-05-16 21:57:33
Current #links: 1366
Current last img link: https://i.redd.it/sfwvs6dawo311.png
It's creation date:
2018-06-13 03:43:12
Current #links: 1593
Current last img link: https://i.redd.it/506fmb9r3i911.jpg
It's creation date:
2018-07-12 11:17:05
Current #links: 1827
Current last img link: https:/

In [251]:
redditImgLinks0 = pklLoad(f'{dataDir}fChats0FromReddit.pickle')
print(len(redditImgLinks0))

16547


In [252]:
capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"
browser = start_chrome(headless=True, capabilities=capabilities)
redditImgLinks = getSubredditImgsLinks(redditLinks[3], browser, "fChats1FromReddit")
browser.quit()
# comment this at the end

Current #links: 193
Current last img link: https://i.redd.it/gm24u3sa4gyz.jpg
It's creation date:
2017-11-17 02:22:43
Current #links: 399
Current last img link: https://i.redd.it/irocl81jaoc01.jpg
It's creation date:
2018-01-27 20:49:16
Current #links: 634
Current last img link: https://i.imgur.com/UPbTfuT.jpg
It's creation date:
2018-03-30 10:18:44
Current #links: 873
Current last img link: https://i.redd.it/2mc0hytxafx01.jpg
It's creation date:
2018-05-12 13:02:21
Current #links: 1105
Current last img link: https://i.redd.it/3t2w7bi8kn511.jpg
It's creation date:
2018-06-23 13:53:25
Current #links: 1345
Current last img link: https://i.redd.it/m825xn53mfc11.jpg
It's creation date:
2018-07-27 06:06:54
Current #links: 1573
Current last img link: https://i.redd.it/jtn1w7fybjj11.jpg
It's creation date:
2018-09-01 02:06:09
Current #links: 1814
Current last img link: https://i.redd.it/yc8d95f6s2r11.jpg
It's creation date:
2018-10-09 10:03:58
Current #links: 2046
Current last img link: https

In [253]:
redditImgLinks1 = pklLoad(f'{dataDir}fChats1FromReddit.pickle')
print(len(redditImgLinks1))

8435


In [ ]:
#redditImgLinks = redditImgLinks0 + redditImgLinks1
# code to save redditImgLinks into 6. fTxtMssgs

# comment this at the end

# Scraping Greeting & Miscellaneous Images
These are images that you see from your family and relatives which you don't need to store for a long period of time :] 

In [197]:
prevUsedTerms = forceLoadPickle(f'{dataDir}prevGoogleSearchTermsDict.pickle')
prevUsedImgLinks = forceLoadPickle(f'{dataDir}prevGoogleImgsDict.pickle')
allGoogleImgsLinks = [] # same as prevUsedImgLinks, but a list instead of a dict
baseLink = 'https://www.google.com/search?q='
imgParameters = '&sxsrf=ALiCzsZLmSQIUVB3D-f2Eqdfn85JCe2tDQ:1663483629074&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjkuLO23536AhUegf0HHWbuAEUQ_AUoAXoECAEQAw&biw=1536&bih=722&dpr=1.25'
extraChip = '&chips=g_1:'  # new link structure should be: baseLink + searchTerm + extraChip + imgParameters 


In [198]:
def getSimilarTerms(html, numOfSimilarTerms = 1000): # 1000 is an arbitrary high value
    soup = BeautifulSoup(html, 'html.parser')
    soup.find_all('div', 'tzVsfd PKhmud sc-it') 
    cardsText = []
    i = 0
    for div in soup.find_all('div', 'tzVsfd PKhmud sc-it'): # If this doesn't work, inspect chrome to see new class name and replace it with this one!
        if i == numOfSimilarTerms:
            break
        if div.text not in prevUsedTerms:
            cardsText.append(div.text) # don't add this similar term to prevUsedTerms, as it will be added in the getGoogleSinglePageImgsLinks()
    return cardsText[:numOfSimilarTerms] # note that [:num_bigger_than_len] will return the full list, not an error, and [:0] will return []

In [199]:
def googleScrollTillEnd(browser):
    scrollTillEnd(browser)
    time.sleep(3)
    try:
        click("See more anyway")
    except:
        pass
    
    scrollTillEnd(browser)

    try:
        click('Show more results')
    except:
        pass
    scrollTillEnd(browser)
    time.sleep(3)
    scrollTillEnd(browser) # here, the browser will see "Looks like you've reached the end" above the page's footer, or infinite loading svg

In [195]:
def getGoogleSinglePageImgsLinks(googleLink, searchTerm, browser):
    similarLinks = 0
    if searchTerm in prevUsedTerms:
        print("Already scraped images related to", searchTerm, end='\n\n')
        return []
    prevUsedTerms[searchTerm] = True
    print("Scraping imgs for the term:", searchTerm)
    browser.get(googleLink)
    time.sleep(2)
    googleScrollTillEnd(browser)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    links = []
    for img in soup.find_all('img', 'rg_i Q4LuWd'): # If this doesn't work, inspect chrome to see new class name and replace it with this one!
        link = img.get_attribute_list('src')[0]
        if (link is None):
            continue
        if (link in prevUsedImgLinks):
            similarLinks += 1
        else:
            prevUsedImgLinks[link] = True
            links.append(link)
    print(f"Finished! Found {len(links)} images!")
    if similarLinks > 0:
        print(f'However, {similarLinks} of them are duplicates! (will be ignored)')
    print()
    return links

In [200]:
def getGooglePagesImgsLinks(searchTerms, browser, numOfSimilarTerms = 1000): # 1000 is an arbitrary high value
    for searchTerm in searchTerms:
        googleLink = f'{baseLink}{searchTerm}{imgParameters}'
        links = getGoogleSinglePageImgsLinks(googleLink, searchTerm, browser)
        allGoogleImgsLinks.extend(links)

        similarTerms = getSimilarTerms(browser.page_source, numOfSimilarTerms)
        for term in similarTerms:
            googleLink = f'{baseLink}{searchTerm}{extraChip}{term}{imgParameters}'
            links = getGoogleSinglePageImgsLinks(googleLink, term, browser)
            allGoogleImgsLinks.extend(links)
    pklSave(allGoogleImgsLinks, f'{dataDir}allGoogleGAndMImgLinks.pickle')
    pklSave(prevUsedImgLinks, f'{dataDir}prevGoogleImgLinksDict.pickle')
    pklSave(prevUsedTerms, f'{dataDir}prevGoogleSearchTermsDict.pickle')


In [191]:
searchTerms = [
    'صباح الخير',
    'جمعة مباركة',
    'عيد مبارك',
    'عيد سعيد',
    'رمضان كريم',
    'عيد ميلاد سعيد',
    'نكت'
]
browser = start_chrome(headless=False)
getGooglePagesImgsLinks(searchTerms, browser, 5)

Scraping imgs for the term: صباح الخير
Finished! Found 37 images!

Scraping imgs for the term: دعاء
Finished! Found 38 images!
However, 1 of them are duplicates! (will be ignored)

Scraping imgs for the term: حبيبي
Finished! Found 36 images!

Scraping imgs for the term: جمعة مباركة
Finished! Found 37 images!

Scraping imgs for the term: صباح
Finished! Found 37 images!
However, 2 of them are duplicates! (will be ignored)

Scraping imgs for the term: اجمل
Finished! Found 33 images!
However, 6 of them are duplicates! (will be ignored)



In [ ]:
allGoogleImgsLinks = pklLoad(f'{dataDir}allGoogleGAndMImgLinks.pickle')

In [20]:
sampleImg = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoGBxMTExYTExQYFhYWGhYWFhkYGRkaGhoWGBkaGRoYGRYaHysiGhwqHRoYJDQjKCwuMTEyGSE3PDcwOyswMS4BCwsLDw4PHRERHTIoHygxLjAwMC4yMDAwMDA7MDAwMjAwMDAwMDAwMDAwMTAwMDAwMDIwMC4wLjAwMDAwMDAwMP/AABEIAN4A4wMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAABQQGAQIDBwj/xABBEAACAQIEBAQEAwYEBgEFAAABAgMAEQQFEiEGMUFREyJhcTKBkaEHFEIjUmKCscEzktHwQ2NyorLxJBUWU5Ph/8QAGgEBAAMBAQEAAAAAAAAAAAAAAAECAwQFBv/EACgRAAICAgICAQMEAwAAAAAAAAABAhEDIRIxBEFREyJxFGHB8AVCof/aAAwDAQACEQMRAD8A9WFdoF61xtUlRYVKBvRRRUgKKKKAK0tW9FAFFFFAFFFFAFFFFAArNa3ovQHSsVqDWwNQDNZrF6wWqAbUVpqo11NA3orTXWwNQDNak0Ma1qUA1VkNWKKkG2oUVrRUUCLEtzUquOHHWu1SgFFFFAFFFFAYpVxPm/5eEuPiOy/3O/09yKa1SfxIxKMEUNuhbUN+una/LofmLVh5GR44NrsiTpEnCZ4uNw5iYlXYoLqbHd1/19iAeViBaYGBVSp1KQCpBuCCNjfrtXkWR4zwTJKeUEU0vzWNlT/uda9A/DnG+LluFe9yIljPvH+zP/jWfiZHOHKXfREXaLFRRRXWWCiuGHxAe5XcBit+hI56e4BuL9wa70AUUUUAVgGs0UAE1Dhx2qeSHTbw0ie9+fiGQWt0to+9c838UKjRAnTIrOqkBnjFwVGogXuQbXF9NqRZjDipJJZo1eJZRDEAUDyfsy769IkUIpLadySewoC1sQBc7W337VxwmNilBMUiSAGxKMrAHsSp2qupkOKGI8fxINRLFmKyeZWUL4JTVbQtrjzc+nO7HJsrZZTPIUDsnh6I4/DXSGvdhrbU3Y32B9aAmS5tAsghaVFlNrIWAY35C3c9q3wGM8Qy7W8ORo+d76Qpv6fF9qRlMRj4T5oo4ZCQBpd5QqsRfVqCrJtfkbH2rri8rxKyXhmUI8vjWMZazaLFXYSC8Zt0F7kVAHs86ru7KoJsCxAue29aNhxqMg+PTpBubWvfly51X4uHMRrLyywznToBlid9KlmZgqmS2+q178lUdKd5PgvBhSLUX0DTqO1/l0A5AdgKA45M0wDLOyl73ABBOnvt0pjVTj4cxAx3ja18PWX1XOrSf0abfy9rfSrbVIN1TRCCiiitCTSIWFb1gVmgCiiigCiiigOZcXtfcgkDrYWubdtx9RXnseQTTygSK4QkavKwOnqSSBvz2/rTPjXM/wArmGWSltKO2Jw8nbTKItN/QOEPyrj+NGaTQYJRCzJ4sojkdSQwTQzWDDdbkDceo61z58EctX6KyjZ2474TjbATJhoVR7ITpWzMiOrso+S3t1tUD8GZicHPhW5wyMAP4JVuP+7XVQ/BvNZ1zEQ+K7xSJKHUsxXyjUrgNyO1unxVcOFlOGzrFQEWWePxE9dDBlt/K7/StVFJUiw6/DDNxictw73u0aLDJfmHjATf3AVv5hTrMnY2iQ2aS92HNIxbW49dwo9WB5A15bwPj2y84mWxaFMXJBi05mNb/sZ0A9dSMOu3avS8jxCThsRG2pJDpjYcvDjJUW931t/MO1WAxijVVCqAFUAADkANgBW9FFSAorjip1jRpHNlRSzHsqi5P0FbROSoJFiQDbtccqA6UUUUAVqJASRcXFri+4vyuPkfpSvijNGw8OtAussEUuGKKTc6mC72sDy5kgUjizk4cs8p/azRYQr41oxdi4csVWyqhbew2AHvQFyoqtYHjOJxFq0hmZ1l0sWWMJqAcsB8DELpJtfWKkZTxEJ3jVALsJjItzqQI4WMsNtOoG+49qAnwZTEkpmRNLtfUVLAEnmSgOkn1IvU6oMubQKniPKqpqaPUxsNasVIufVSPlSbhvN5CuJeUKY4yZQ8QurAhmcKb+e2kb7Hzb2oCyM4BAJAJ2AvzNr2HfYE/Kt6r3EGYRo2Cmc6E8RmN7bBsNNYe9yB86h4LjFZVZWlggfTG6MW8VdL3ujLdLSLaxF/1CgLbRVayzihSDHIyvKJUiTQrJ4iOUAlWNrkKAzX5jyHferLQBRRRQBRRRQBWKKh51jfAgeX90D7kD+9Q2krYJlZpDwvxMmKFiNMg5jofb/fLfuA3xmJWKN5G+FFZ29lBJ+wqsJqStdBOyj/AIt5Wk8mXxuxVZJ3hLDmDKnlO/8AEorHDmbJi45MpzRQcRGfDOo28YLuro3MSAWNxudmHUDr+IwlxmGwUmBUyu2IiliK/CAsbuGduSqCBe/tz2qRx7wV+eRZo7R4pFHXyuBvoLdCCTpb/YsCfwvwLhMA7SQK5kYadUjaiqkglV2AAJAv1NhvSzjhVw+OwGL5XlEEh/hkvGL/AP7Cf5aqOU/iLjsFIcPi0Moj2ZZfLKvYiQX1j1IN/wB6pHG/4gYTHYJo1SaOYMrR3VbBgeetWO1iftQE3CXgzLOSUVsMsJknDDYu0YkVexJ1S39/Wk2IwmY5GUaOUvh3Cm5F4y5Fykifoa/UEXHXmBvxX+JOHnw00MGHZZMSEE0jaVuQFUmyklvKukctqafhbnc2Plxj4xxJEIolKOAIlUlyfIfKBZTcntuagDDJfxdgcAYiJ4m6snnT+zD2sfepeN/FvLY72aWQ9kjI+7lR96804rTC4jE+HlcEpG99OpxIdt4ozdlUd72N+QG5l5Z+E2YzAM4jgB6Sv5rd9MYb6EipsDPNPxBlzPEYfCQxmKCSaISAsC8i61JDEbKlgSVBN7c7bH2M15Twn+FuJw2OgmkeJ4oizsUZtWoIwUaWUfqKm9+lepLiUJ0h1J7Bhf6UB1oooqQFasARY7g8wd6hZ3j/AAIi9xe6qoKs2ok/Cqp5ma17AVFh4gjYYcggiewDBkUBtrroZtRNzawBI5UA3UAbDYem1Zt1pHheJI1jLTuEPiYhAArGyRTNHqIANgF0XY7XNNfzsevwhIniAatGoatPfTe9qAVtkF5ubfl9Ql8PxG0+KG138PT8OuzW1Wv0p0RcWO/S3pVcy/iCV4HkIXUuDhxGw28R1lLdfhui7e9O8NitQ8wsQiMTsF84PLf0P1FAccJk0ETB0SxAKrdnYIp5iNWJEY9FAqWdCW+FdRsOQux3t6mwJ+VQpsefGiiUWDNIrXHPTGHBXfl5h9DXDNZkZ8KQQ/7fQCrmynwpCfhaxPltY32J70AxxeDWTTq1eU6hpd039dBFx6HapFRsymKQyOvNEdhfldVJF/pSnLM0nmnCExxqqo5QhjJIpQHWpJCqmo2uNR2sbUA/ooooDVSCLitqgZXPe6HnzHtU+iBiqVxNnrlpMPIAEJsDboGuPnsPryq60rzfh+Kc6muG5Ejr7iubyYTlCoPZWSbWip/h/lbjEtJ+hBa/Qlgf7W+9PPxOxJjyzEkAksnhbf8AMYIT9GNMciy5MOZIkva6vv6rp+XwGmE0SupVgGVgQwIuCDzBHUVPj4vpw4kxVI+c+GeMMZggUgmKoTfw2AdLnmQrDy/K16vPAv4j4vE46KCdo/Dk1qdMek6gjMtmv3H3pB+KPBH5KQTQgmCQ2F7nw356GPY9D79t3OY8KYbDYSHN8G0pKNh8RoYoR4ZddQsFBuNW+/Q1sSX7jLLMBJA0mPRNEY/xDdXW/RHWzXJt5RzPSvAsekcszjCRy+GblEfzyaQNydI5deth1NXD8R8/lzXErhcFqlhiUvZeUjhdTSG/RR5Rfrq7im/4N5hhPys0Sqq4kLIzsfikjIJUof3VFgVHv1oClScDYhcB/wDUWKCIiNlUElykjBQxAFlHmB58qfcP4BcZDHl+ADRK6Ry5jMxYhnXYJufhvchVsDcdA1XbBRoeHlEuy/kLk9R+xuCPW9rVU/wMz3wppMFIAPFu63Fj4qCzIfdQTbpoPepB6PkeTYTL0SGIBDK2gM28krhWfdup0oxtsBY2pTn/ABqMO+P5H8rFhtCnkZ5jJYG29t4r+l6jcd8RLBmWVxMfKJHkk9PERoIyfS8jm/pXlXHWKdsxxlyd5nVgDsRGdCXHWwUUBnCcb5hHOZxiZC7G7KzFoz/D4R8oXpYAW6Wr2nhPiiLMMKZfKjoCJUO4jexs2/6DzB9x0NfO5q4/hBnZw+PRCf2eIBhbtqO8be+oaf5zVWrVMHoXDqYnCzNJii6xEFWLEuGckaSLE9jvV4BvUF8P4kckRNip0qedtgyGx52uB66aQ5ZxneyzJZhs1trEfY/YVzRcfH1KWn1ZW+JZsZg45V0yIHW4NiOo6jsef1pC3Bi+QDEyhYj+yUCKyAOJAP8AD8wDKp3vyqxQyh1DKbg7iuldhYrU3Byuul8RMV1PJa0Q88hu5No91JJup29KyeFX8TxPzUmoOZdXhw6/EaPwydWi2nRtp01ZKKAU4Th+JEaMFirQx4cgkf4cYcA3A+Ih2vS3H8MS6WCTzSFwkcgZ4kvHHcp5hCbEE9Oeo3q0UUAik4bup/8AkTeKXZzIPDDeaMRldOjTbQq8he4vejLOGliIZnBKukihECL5ImiF1ubkqxJItuB2p7RQCHGYDEf/ACEj0MuIJOt3YGPVGsZGgKdQGm4sRzttzrvl2ReG8byTSStEpSPUI1VQwANgigk2AG5NNqj4LGCTVYEMjFGU8wR/YixB9aE0+yTRRRQgr+HJ8RLfvD+u9WCq9hj+1T/qFWGqxJYVzlfSpbsCfoL10oqxAi4mklj8KWKREVWtLq3LISDYC2/Jtue+1OMPiFkUOjBlPIikWcYZZdOGdxG4OqFj8LqLjSLnd1GxHPkeptMyvJFgjCh21C5LCw3Pobj61yXlWV0ri69ld2d8/wApjxWHkw8nwyKRfqp5qw9QbH5VWuAcsMuTjCz9fzWHb5TSpt7dPamPFvE8eXQs8j65SP2UZ0hmbvZQPKOp9O9SuCcO8eCw4k/xGTxZNrftJWMr7dPM5rqLHnX4ZeHluEx2OxKHVFJ+WYCxY+GVDItyBcyPbn+iqlxjkcmX4kmIlY2Blw0iEi8TjYAjqAdJHb0Neh/jdgNGDQxjShxBklA5M7q1mb5g/MimfD+UQZllWB8a7eEsJDD4tcJCOtyOTaCpH9wDQEvPoBh8vw+HGw14DDG/7pmiRgf5Q31rzn8VMrfAZiuLg8gmbx0I5LOhBkB7gmzEddbCvR/xTwMkuXy+ESHjKSi3P9k2okeo+L+Wl/EGFXOspWSIDxbCWMdpkBEkfz8y/MGgPMuK8dLm2ImxUUR0QwRs6nmkaWD+41u5/wCkE9Kr/hySB5jqYKy+I5N/NJqKlid7nS2/p616J+FGMhwWBxeOxAOkyJABa5fStwgU9S0hBvsLb8ql/h9mWU4hZstWCSL8yXa0hU6wLsqI67qYwLrf90m5NQDyljUvJCfzEFufiwW9/FW33p3xdwHisFIR4bzQ38kiIWBH8YUHQw6327U1/DXg6UypjcRGyQwsGiRgQ809/wBmqqRfTqsdXLYdLkAewS4sJ+YkNrKVUdiQinf0u1vkaomRYQYyRxfS4LMpO11vsDz3t/SrlnmCb8o6DdrEt2LNcsfbUxPsKq34cYJzM0liFUMN+5FrfX+/auHyVyyxg1aZSW3RccFF+WhCu2rcAW9bCw78ifmaYK4N7HlsfQ0sz+cJ4TP8Akux6CwNr9hz3Pp3rtluMVw8gXQhNwxGnWbAFvXkBf09K7opRVI146tE+q1xLxJNA5iSBmLBfDbmrEmzAjoRW0XEIRGWRgZBfwy/lEgvZT8ztsNjftXeDiKEi0pGsH4NJ1Ajl5DuDVrRrjjxdyVr4GGODNh3udDFDcg20m29jXbBC0aebV5V3PM7Del82IkeOR2UxxhGsGFmYkcyOaj33rbH5h+Xw6toLvaONEGxeRrKq36b8z0AJqDNp9L5GtFVybNMdEuqWCJ9VlXwnbyO2y+JrAul7XZeXO1bYbNpItbTypLGsTTa0TSF0EhltqNx267GrE/TkP8AVSPI5teKxhX4Q0S/zqhDfSq3i8xmv404DYiPRHho0JA8bEAtpYdSqFL9Kt3DeVfloVQnU5u8rfvSNux/t7AUNHFQi7e3r+RpRRRUHOVaZypBHMEH6b1ZMPMHUMvIi9VnFVwy/PjA2lwWjJ3A5qe4/wBKzTotVouNZqNgcbHMuuJwy+nQ9iOYPoakVfsqRM1y2HERmKZFkRuat3HIg8wwPIjcVQuIuD80hU/kMdNJF0ieQiRR2SVj5re6n3r0mtXNtybAbknoKkHjnDXAk/i/nM0PhQReeQzSAu+nkGJJst+dzvyA3prn/wCMSqxTBxB1G3iyXAPqsQsbe5HtXbGZfPn8hbxDDlsTkREDz4h12aQA7aQbhSbgdib2f4b8M8rRNH5YP3Z3kLH11atvlaoAiy7NJ85y3FrLEislhEyagHlQeJYKxNreUXufirb8Cc0D4WXDX80MmtR/y5fMD/nEn1FXbIsmhwkKwYdNEaliBcsbsSSSzXJPv6V5fmCvk2btIoth8WJNJ6DXuR/JLpP/AEsKh62D0jOeIYYbqxBbkQeQ9DYEk78gDzF7XqmYTC4rLWbE4GI4jBSkvNhlN5Im2u0X74tytzXTccmCHHTF5SSTtc779TYevX3JJq3cIZ4IIrSXIOwtb9JO+5HQgfyV5+LzVKbUtIzU9i/PkwWbYMwZfLHFOsxnML/s3MtmVw6He51HzC4uBUb8Nvw1xGHxS4rFBU8LV4aKwYszKU1ErsFAY7cySOVt7s+CwGPF5cPFKwH/ABEQuB3DdvUGtoODsGmyRuo/dWacL/lElvtXoRakrRoT8ZmcURCM15G+GNPNI3S4Qb23F2Ow6kVrhsKzOJpfiAIRAbiMHmb/AKpCNi3QbDqW6ZflsMAIiiSMHdtKgFj3Y82Pqal1YGGFcsPh0QWRQBz27+tdqKiga6t7da443DRyLaQXAIPMjcexHc7etRYJAZ5ST8ARBv3AY/1rlnjQGNWlXxVv5VuSGNjzUbMLA7EHlQuou1Q0jtYBbW6W5W9LVkMDsDy9aiRNEEWNdKKVsqL5bKRyAW2n5Usy3FYaJHkhhCklVuB55NiVu53bruTSwoyfoeSxKw0sAQbXB3GxuNvcUh4vlZREyqWeKRZkX/8AJo1CSNf49DMwHW217GuiZliJVBQxwjUVJkvIdQNrKqEBh66ue1MpI45g0b2e2nUPXmCOxuLgg3FqklJxdsreccR4fERII50RT5mLbMLD4bc7+1KIZ43T9QwkZV55WuPFaMllw8QNiV17nvv3plnXDQR0MZ1mR9PnhjkK36lyBf8AmudjvtTTB8KrrV8RK07JYopAWNSOREY2v9vSiVXs6+eOOOk3XdFcwWHnaaSUqPHSFsRGjWsJsQSovcj4I0RKvGUYszQRSkWMkaOR2LKCRSLjXAxkxyNI8RP7JygJLxnzFTYjluevM7GrHholVFRBZVUKo7ACwH0qWYZZKUVL+o7UVrRUHOVPFHakOZGn2KpHmCVhPo0gdeDM6EExRzaOWwJPJXHwk+m5B+XavRDXjkyVZOFOK5EaOGTzoxVFYnzLc2G/Ue/1qMc60y04XtHoFQM6wrzQtCu3iWRzexETG0hH8WjUB6kGp9FdBicMHhI4Y1ijUJGgCoq8lUcgK70UUAVXPxC4aGPwjRj/ABU/aQn+NQfL7MLr8welWOigPEOGsLLiFjKgkkMjX5q8ezqw56xcHTzsw705bLHLaTyFlVbj1Pmsb6ySSV6XtbanHE+WjBYh8WAfy2JsMSACRFiACI8RYcla5RyLfGT1qZhZyrAjaQ2KSWsJOwcjqeRU+tt+XieZhx4mrupP16M3BDDhHI/BXWwsT8K9gbXJHQmw25gc9zYWGuWFnEiK45MAw+Yvau1evihGEUo9F0qQUUUVoSFFFFAV/iHJNbCSJGLuwEmmQoLAWDspOk2sASAWtawNhbeHhWLSBI8jG2+l3jX5BCLD3JPcmntFRRbnKqsrAyvCeL4IXElwbFw2IKrt8JlvoHt7VNHCuH/5tuoM81v/ADp1RU0Tyl8ib/7Vw9v+J0sTNLcW7HXUrKMtEAZQbguWB3vY2sGZiSxAHO/yFT6xSiHJvtmaKKKFSBnGXCZAhZkswa6+n+/rapqC21bUUJbbVegooooQVPEClWLipzMtQpo6ykjSLK9isPUB0I5VYp4KXYjC1jKJrGRfeGc5XEwhr/tFssg7N39jz/8AVNq8nwWIkgkDxtpYfQjsw6irllfG0TgCYGJupsWQ+xG4+Y+dbRyJ6ZlOFdFnoqFhc2gkIWOVGJ5AEX+nOpta2Z0FFFFAcsRAsisjqGRwVZSLhlIsQR1FqrmAy1MMwhnXXCTaCVidh0in6FhyWRviAAPm+K0VqeW/LrVXFPsBHGAAAAANgALADsBW1aoANhyGwrarAKgLm0RIW5BMrQAHrIisxHtZTv7VLlvY6barG172v0vbpSOHhSMkPLoeRmd5j4aEOztqtdgWVV5CxvagOOZZrKsWJilHhyeDNNEVkD2VVC2BCqVIJUi9+Z32refFP4WIbURbExKDc7LqgBA7DdvqaYjh3CbD8tFsbj9mvP6VLkwkbKysilXvrBAIa4sdQ68h9KAVYnMNU8Kqw8uIkiYLccsM72bvuQe3Kp2AzJZWlTk0TlCpI1WABDW5gG+1YwuTYeJg8cEaMBYMqKCAee4FGZZRDPYyRoxUixZFY2B3W7A2B5UByTO49MjkELHMsFxYgsxjUH0Gp7H2NReHsyeSfERSCQNGwZddvgZmC2si2Hlv+rYjfnXfMMmWRY4VWNIFYNJGF+LSQyoALKFLAE97W61PweBihBEUaRg7nQoFz3NudQCPlGcw4kMYmJ0EXuCpsfhIB5g2Nj6UwpZk+UCBpXJUvKQWKRiNQFvYBLnqWJJJJJpnUgKKKKAKKKKAKKKKArjrUeSOppWtGSqNFkxZJDUaTDU3aKuEoUcyP9/+6o0W5CWTBXrkcB6U6JTv3+1Z0ptuN72+VVpE8xKMD2501wub4mMWD6h/GNX35/euw0d62ijDEi3L/W1SqXTIckznJn2Jba6r6qov971Jy7N8QuzEOP4hv9RQuFFdo8PVkmQ2iambuf0D6mgzO/M/Icq5RQ1Lijq2ytk2JrgGt64RG1db1cgyaipmEbAFXU6h5d9zdPEAtzvp81u1SHUEEHkQQfnSrD5RIDCHlUxwEMgEel2KoY11vqI2Um9gL+g2oBZl8sjfk2nll8KSKHw9D2DT6NbCYjzte1xvp2IPq5xb+Kz4dZHjZRGzslr6XLeVSeR8u5tyP03w2T4eN9aQorC9iF3F+en935VtjspgmIaSJHYCwYjcDtq529KA0GZxpqR3JaLwVdiObS2VTt3P0rmc0dZlikiKrIWWNw4bUVBazLYFbqCetcMRwphjbRFGlj5rRq2ob7HV78+dTsHlEETao4kVrW1ADVbtq52oCRiGYKxUamANh3Paq9nmXYnFQx2tG4LakJKgi9lbr0HI96sM0Wq25FiDsbXt0PpXWqSjy0+iGrIuWQPHFGjtrZVVWbfcgbm53PzqTRWaslRIUUUVICiiigCiiigK/PilVglxrPJb7+9u1Goj4uXel2AywpI0g8zi1yTzJvcX9v604QhhccuX+oI71hjnOauSolnMpWjwE9bfIV2ih07C9ugPT0HpXTTWlX2CGcMf3vsPp7Vn8s3732FSwtbBKcUQRVw7fv8A2FZiw9id739AP6VLCViSygsxAUAkk8gBuTUtJbByWKoGIz/DR7GVSeyXb7qLVVuLuKTKfChJEZ59C/cnqF9Pr2C/IcCZ5UjHU/QcyT8q8zN5zUuONWUct0enYU6lVradQBsbXF9wD62qSq0kzFTJMsKn4Uvbtva5p1h4tKgEkkC1z1r0404rey/o6VsKxSjM898KQLYEb3BuD0tY/XpVMuSOKPKXQHN6L0twOdxSHTfS3Ox5fI1wxfFWHQXD6z/CNv8AMdrVmvKwuN8lQHFbAUgPE/lR1hLK5AGltRAP6iNPIdd6ynELatNlJ7b7e+9dEfuVosotj+1ZqPHigIw8pVO5JAH3rtBIrqHU3VhcEdQeRqLV0VNqKzYDrXE4uIAnxEsNz5h/rU2gdaxaqnmHGvm0wIGUG2tybH2A/wB+1SMq4u1MEmQIT+pfh99+lY/qMfLjeyaLMFrLCoWaZrFAuqRrX5Abk+wFUzO/xBY3jgTRfbW1iwHUhBsD7k+1VyeTCDpvZVtIvtqzXPDRqEULuABYk3JFuZJ5n1rrauhO0SYoooqbAoSIC9uu5rSFfO9uR0n57g/YLXSV7AnsCaIUsPU7k9zVAb2otWiYlCxQOpYcwCLj5VmaVUGpiAO5qOcauwZZqVYvMZFdY1OrUbXsASdtuw2P2rD58hfSEJX964H/AG//ANqNmOJCjWu+llcfWx+xNc/6mMk+Dtokl5XiZfzDRNyAZulrbb9+o+9duLUY4aUJuQFJA5lQwLfa5+VKMqxhWSXEP8VliUdLs2qw776RTlgYomI8zuSzEnqRbnvyAqZZPqxcfyJOzylPO5bpyHtVu4fkXD4d5NSiZ1LIDz8Nd9h1JsbDqBSTF4LSxB0qS19z5AOe4tv7WrbBYJ3bxQ7WIurG926A25BbCwHavNwYX9T5ropGGy7ZHi1QtLJvLMQFUbvoGw26C/8AQGp2a8RRQi2zuf0gjb1YjlVJgSaeVlC2C7E26dPc7VYMvyQJva59a6MWTM7ilX7/ALkpDHE5+whDmIqzjyA7i/S/WqhxHjA0gkJ02Og77aiOX2P0pxxHjBoVVbdJU1DrpKv/AHtVex7o+FiB3fxXlO3MKGH9SDWufLCUHCaul2W5JIWZ3mOgBFa5cgXHbmR9K1nxemME9R/WoqpC+LjEr6UsxO/622AB6XJvvXfMcAHDjXpFxovYmwBvsOtz9q4sfhrIk4ql+Sii5O0TIMTBiYIYwWiaOUlgLiyhtJGqwuWNtx1vUjh7OtLhifOYiL2uA4Jfcdbkqv1pX4IUFFjY2BDdF3tcbnmdjt87VEy9jE4VxbUARfseX9K28jJNdevgmbaG+d5xJLLqka5uAB+lR2UdBXpnBeK1YOEnoGUeyOyj7CvHZZh4m/7xp9wvxY+HkGHY3jcm1+auRe4PYkWt6373y8TI1kuXtFIsx+JXGcjYg4dGKxpbVY878vmQL37H0N+PDGCfEQl5IGV76QzSlo5Ft8YjkJKqee3MWsa7rkcWIkfETIBDGUJBsTM0cMYGrqFU6gR1t6mnePzQIFXSFLAknYKLWtqPMX2AsCSehr0XHn2ehg8d5Nvo6PkpYXVlU+oNvtXPNsvaKLXcNoIuw2A9weV+XzpBiMxe+pJgyg+YAlWFz0NirD3tTXLOI2hdVlI0ybK3O/PZhyItWcvEj/qd2T/GtRuDs5STCb9R1WAsbnlyAPaoYwgVruBubfT/AN1F4k1Q4qIwoEEwb9mLldSMLsDewVgy7CwFql5jjV0r4ibHmyndflbdT35iuHJDjdrZ4+XGk2/Z6FkOcCRY40BOgKrs2w2T9Pfcc6dawdgR9a86m4si0RKF8OzgkputgGDA23tuD/6qzcMosg8ZHuLkCwt7jffn6V62DJCUVTI0x/es1i1ZrYqKWqLiZ2QHRv266fW36gO3Pa3t2kNLcdNaqS6LIhHK4lQskjM/Mkkc+fIC4PWomOMh0s5LECxPb/feu+HmQveTpexA/wB7VIx0YAI6V43lRUXrSZevkSqx5iuuHzBidNr258rgf6VkRKtx0qM0AV9QvuLG1uVcUJvHK4lXRYeHtEhU7XTzWNja4tfbbVy+9T86JKMqsVJBGoc1vtcX69qr/A+LKiVGIDXBA3JNiQT67aTb1PrUjPZ9XlLfK21evDKo4XJdkNCA4CGIlJAoR2GgjewCj4id7kljc3vtfnXXN33WOMlYlA0gHmRffvbt07d655xFdO9RYY2IG+4HMc64JZpSi1038BxOkckkfmRiCOxsfrTPL+MJAbSnWORBAVh7ED+tIMZjXh3caluR2O3r1rj+dil/S4Pe3+hrKEsuPabMXcRzmmLSckpsemrY39LAg+162wmSmQsocLoVRYDcKb/qPUlT/u1QMNiLCwuPemOSYxhMApuWV7qTzAF/rcD71pDyFOa5IsnZX854ZdZNZOpdum4t0IG1vWp/DGWKMQJJFNibKWBtckkC/wDvf3p1FjPGLAKLDc3N7/UCleOzNAdCk7He3QjkDWs58ZfbtItx9olS44anDLYhiun6G/3qt8VkMFdAdS35duf2t/WpOKxBB1c+9R3lV1Nj/qDXLGcufL0UbbEE2M1kOOfPbvU7h62Ixqh/0RyzEcxeNCwuO17H5UuziA28SIgMNnUW59wO/wDWlXD2cthMVHOQX0Mdan9cbAq6n3UmvXxYk/uRZRovGQcTxOrwgst5LBXIDsundmFzvfVf2p1xDOI0MhI0TbKSRZZEJYKx6Ai4PaxPQ0n4g4XjliOYYK0yf4gdG3Vuf7aIC6sORN7X3IAvaVkOOhx8LYaUFWLeZH2bUpGwPMOOh6jYiurG9VVfk9Xws7S4+zTF4FIVjcLZhI4e6kllIJIc2uBy7jcWo/JqSP2ZAPn57Ag8/LzPyprNwwEXRGgRBuigt8VramZrG/12v3NcMDkvgMHJub3CgCxPew7XrRqlZ7Ec1RtsZ5jCAIdQOpjoHM7aSx9gN/8AN61OxHCryRhrAqQLaWufY+vteomISR31uCAim1+lxcn7D6VxyjPZRojEhALoACbgeYW+V7V5mSUOX3Ls+eztOTZwn4Z8JHl5FQX/AMoJtbrcbf7tVyyHKZsNoGtXjPldQHBQtc6h5yrDUbHyjbfpUDiXHR+H+X8S8jaQxAvYE7k9BexrTJs1lWZAWZkdtLAm/wAW17dNyKq8uLFkS9v/AIcmrLres1Gkx8akguARz3rNeryXySKpDSvMRzprIKhYpL0krRMSsSyFTcVPkzZGQXXSRzA5fKtsVhQagy4ewrhzYVPTNdMXYnN4zKE1FGY6elr8gCT1JFvnRjnI73qo8TQHxHN9r8qmcO4yaVlhZ9Q5Atuf81rmuDLguOjCe2X/ACzK9o5Te9gTbnfry9ai51fVff6Ef1q04LD6UHsL+9RMwwwIN69CHjKOOkXUipopa4BuPWpcMSxjc79uZ+QG5+VVyWYmQsNrnYdh0q5YSIGFHPMrc/Lb+1cGDGnNr4IT2V/NIXlVhpAVWuAfiJO99tgN/eomGwVh2qy4bCgqL9dz7nc/c13GWqa2yeK5u7Dxp7ZX8NgdSM9/hvt12F96MZlllVudgDf3F7inYyXS1w2xBBHcV1XLiikFrqoJAtuABewvXM/ElHolJLoUcNS2aQPYal2Y87joCe+/0qK2T+I+obX3NSYsZqBMaKO2q5/pTThrELKruF0lSFtzG4vt8/6CtcGOM3x9ixTiMg2ueg3pPLhYV3P1vb7CrdxBIRDKRz8N/wDxNeY4zGtWmfDxaUSkmScfEjt+zAt7m/8AWkuKymIsVMln7Cw+3X611wWKKyjqCbEe9ceLsPacEH4lVvnuP7Vth5qVN+iIy+TfBYeXDMJYZSjqQQy+U7HkeeofwnY8qtuK4YkxAGMwTMvigF0G4SQAaomTexXkLixUA3vSrJ8vBRWYlmIBuel+1Scfi5sJZsPM8TXAJQ8xYmxHJh7itHJv2btUrRY8iy3MVA8dQQOt2Qgdyf7Wq5ZXNGqXZCSBuyi97dr2NI+EZcRiIUlxM7Sl7sFsoUC+keVQATte9utWzDwACtoY73ZLyyapsqOf58JLpEhVf1FviPpboP60hyjDAyFb2b4ornbWGBCC/K4vb1Aq7Z5lEb3a1m7jr70ufJotIYi5Hyrhz4cnO/RS7EstotTDZyxNm56j0sf99ae8HYTDSbvIGlO4TUykW7C/mPtVZzxvELMwv78/e/f1pXBiWXcE7cu/1rlTjCdtWYyfFnsjZHAf+Ev0orXhrEvLhonexYruTzNtr/as16yyR+C5/9k='

In [21]:
len(sampleImg)

14175